In [1]:
#### WE BEGIN TO SCRAP DATA FROM STANDINGS, PLAYERS PLAYED, SCORES AND ETC.

In [2]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [3]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [5]:
!python3 -m pip --version

pip 22.2.2 from /opt/homebrew/anaconda3/lib/python3.9/site-packages/pip (python 3.9)


In [6]:
!python3 -m pip install ipykernel

In [7]:
!python3 -m ipykernel install --user

Installed kernelspec python3 in /Users/joel/Library/Jupyter/kernels/python3


In [8]:
!python3 --version

Python 3.9.13


In [9]:
!playwright install

In [10]:
!python --version
!python3 --version

Python 3.9.13
Python 3.9.13


In [11]:
SEASONS = list(range(2018, 2024))

In [12]:
SEASONS

[2018, 2019, 2020, 2021, 2022, 2023]

In [13]:
DATA_DIR = "data" # main directory folder that we'll store all data in it
STANDINGS_DIR = os.path.join(DATA_DIR, "standings") # "monthly" standing pages are stored together
SCORES_DIR = os.path.join(DATA_DIR, "scores") # "monthly" score pages are stored together

In [14]:
#url

In [15]:
# Use Playwright Asynchronously to bring HTML <tag> div which contains the valuable column data target

In [16]:
# Opens a Web browser to communicate within the program to render data from HTML file in the source "Basketball Reference"

In [17]:
#soup = BeautifulSoup(html)
#links = soup.find_all("a")
#href = [l["href"] for l in links]
#standings_pages = [f"https://basketball-reference.com{l}" for l in href]

In [18]:
#season = 2019
# months = ['October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June']
# match months:
    # case 'October':
        # month = months[0]
    # case 'November':
        # month = months[1]
    # case 'December':
        # month = months[2]
    # case 'January':
        # month = months[3]
    # case 'February':
        # month = months[4]
    # case 'March':
        # month = months[5]
    # case 'April':
        # month = months[6]
    # case 'May':
        # month = months[7]
    # case 'June':
        # month = months[8]
    # case default:
        # print("Month not found")

#url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"

In [19]:
#html = await get_html(url, "#content .filter")

In [20]:
#html

In [21]:
#standings_pages

In [22]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p: # creates our instance variable 'p' to use from playwright async api
                browser = await p.chromium.launch() # initializes and launches a chrome browser using playwright async api
                page = await browser.new_page() # creating a new tab in the browser 
                await page.goto(url) # sending given url to the new tab page
                print(await page.title()) # print the page title to know the process of the scraping
                html = await page.inner_html(selector) # only use the selector tag found in the inner html file
                
        except PlaywrightTimeout: # call the PlaywrightTimeout from Timeout imported by playwright library api
            print(f"Timeout error on {url}") #Not successful after 5 incremented seconds
            continue # goes again to the top of the Try conditional loop to continue executing
        else:
            break #when successful it breaks out of the conditional try loop
            # after trying 5, 10, 15 seconds it will return the inner_html selector tag from HTML
    return html # returns html provided from source webpage from chrome browser

In [23]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a") # find all <a/> tags and call them links 
    href = [l["href"] for l in links] # for every link in links we find the list of href individually
    standings_pages = [f"https://basketball-reference.com{l}" for l in href] # for every href we find each path to complete the url links
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1]) # saves the path only after the last "/" route by using [-1]
        if os.path.exists(save_path): # to avoid repeating the url path
            continue # if true it does exist then, the function will continue to execute the next line of code
            
        html = await get_html(url, "#all_schedule") # gets all the table from the div <tag> from source html and provides the individual nba month schedule games
        with open(save_path, "w+") as f: # read/write the html into a file as 'f'
            f.write(html) # writes the html into the file 'f'

In [25]:
for season in SEASONS: # create loop to bring each season
    await scrape_season(season) # its an async function and uses a promise by using await for request/response of each given season while it finishes scraping all data from each data month table found from source 

Timeout error on https://www.basketball-reference.com/leagues/NBA_2018_games.html
2017-18 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games.html
Timeout error on https://www.basketball-reference.com/leagues/NBA_2019_games.html
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2022_games.html
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
